In [3]:
#Final Code for Allison Peteka, Emily DeGeorge, and Stephenie Lin
import requests
import random
from tkinter import *
import json

root = Tk()
root.title("Movie Suggestions")
root.geometry("500x500")

API_KEY = '9194699dfd4d5f613b5dfb17c43caa7f'
URL = f"https://api.themoviedb.org/3"

genre_label = Label(root, text="Select a movie genre:")
genre_label.pack()

# define the list of genres
genre_options = ["Action", "Comedy", "Drama", "Horror", "Romance", "Science Fiction"]

# dropdown menu for genre selection
genre_var = StringVar(root)
genre_var.set(genre_options[0])  
genre_dropdown = OptionMenu(root, genre_var, *genre_options)
genre_dropdown.pack()

# dropdown menu for runtime selection
runtime_label = Label(root, text="Enter your preferred runtime (in minutes):")
runtime_label.pack()
runtime_entry = Entry(root)
runtime_entry.pack()

# dropdown menu for actor selection
actor_label = Label(root, text="What actor/actress would you like to see movies by?")
actor_label.pack()
actor_entry = Entry(root)
actor_entry.pack()

text_box = Text(root, height=10, width=50, font="Elephant", padx=40, pady=40, wrap=WORD)
#https://www.tutorialspoint.com/how-to-word-wrap-text-in-tkinter-text#:~:text=In%20Tkinter%2C%20we%20can%20wrap,WORD%2C%20CHARS%2C%20or%20NONE.
text_box.pack()

def get_movies(genre, runtime, actor):
    # Make the API call to get the top three genres based on the user's input
    genre_search_url = f"{URL}/genre/movie/list?api_key={API_KEY}"
    genre_response = requests.get(genre_search_url).json()
    genre_ids = [g["id"] for g in genre_response["genres"] if genre.lower() in g["name"].lower()]
    genre_ids = genre_ids[:3]  # limit to top three genres

    # Calculate the runtime range based on the user's input
    lower_runtime = max(runtime - 20, 0)
    upper_runtime = runtime

    # Make the API call to search for the actor ID to pull actor's movies
    if actor != "":
        actor_search_url = f"{URL}/search/person?api_key={API_KEY}&query={actor}"
        actor_response = requests.get(actor_search_url).json()
        if actor_response["total_results"] == 0:
            return []
        else:
            actor_id = actor_response["results"][0]["id"]
            search_url = f"{URL}/discover/movie?api_key={API_KEY}&with_genres={','.join(map(str, genre_ids))}&sort_by=popularity.desc&with_runtime.gte={lower_runtime}&with_runtime.lte={upper_runtime}&with_cast={actor_id}"
            #https://www.geeksforgeeks.org/python-map-function/
    else:
        search_url = f"{URL}/discover/movie?api_key={API_KEY}&with_genres={','.join(map(str, genre_ids))}&sort_by=popularity.desc&with_runtime.gte={lower_runtime}&with_runtime.lte={upper_runtime}"
        
    response = requests.get(search_url).json()

    if response["total_results"] == 0:
        return []
    else:
        # Shuffle the list of movies returned by the API
        movies = response["results"]
        random.shuffle(movies)

        # Get 5 random movies from the shuffled list
        movies = movies[:5]
        return movies

def suggest():
    genre = genre_var.get()
    runtime = runtime_entry.get()
    actor = actor_entry.get()

    movies = get_movies(genre, int(runtime), actor)

    if not movies:
        text_box.delete(0.0, END)
        text_box.insert(END, "Sorry, no results found.")
    else:
        # Print the suggested movies' titles, genres, and runtimes
        answer = "Based on your preferences, we suggest the following movies:\n\n"
        for movie in movies:
            # Make the API call to get the movie details, including runtime
            movie_url = f"{URL}/movie/{movie['id']}?api_key={API_KEY}"
            movie_details = requests.get(movie_url).json()

            genres = ", ".join([g["name"] for g in movie_details["genres"]])
            answer += f"- '{movie_details['title']}', which belongs to genres {genres} and has a runtime of {movie_details['runtime']} minutes.\n"
            #will only come up with movies the actor was in

        text_box.delete(0.0, END)
        text_box.insert(END, answer)

search_button = Button(root, text="Search", command=suggest)
search_button.pack()

root.mainloop()

